In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/Tesi')

In [ ]:
import cv2
import math
import copy
import random
import pickle
import numpy as np
import pandas as pd
from tensorflow import keras
from collections import deque
from keras.utils import Sequence
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras.models import load_model
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sequence_length = 4
sequence_stride = 2
batch_size      = 16

# Split of the dataset using 5Fold-CrossValidation technique

In [ ]:
# creating a dataframe with all video names

csv = pd.read_csv('csv/wild_long-range_annotation.csv')

dict_videos = {'Name video': []}

prev_video = ''

for index, row in csv.iterrows():
  # finding DATASET NAME from csv 'filename'
  idx_of_start_dataset_name = 54 # for short-range this number is 55, for urban short-range this number is 56
  row_substring_dataset = row['filename'][idx_of_start_dataset_name:] 
  idx_of_end_dataset_name = idx_of_start_dataset_name + row_substring_dataset.index('/')
  dataset_name = row['filename'][idx_of_start_dataset_name:idx_of_end_dataset_name]
  # finding VIDEO NAME from csv 'filename'
  idx_of_start_video_name = idx_of_end_dataset_name + 17
  row_substring_video = row['filename'][idx_of_start_video_name:]
  idx_of_end_video_name = idx_of_start_video_name + row_substring_video.index('/')
  video_name = row['filename'][idx_of_start_video_name:idx_of_end_video_name]
  
  if (video_name != prev_video):
    dict_videos['Name video'].append(video_name)
    prev_video = video_name

# dict_videos['Name video'].append(prev_video)

df_videos = pd.DataFrame.from_dict(dict_videos)
print(df_videos)

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)

subdivisions = []
for train_indexes, test_indexes in kf.split(df_videos):
  subdivisions.append([train_indexes, test_indexes])

train_videos = []
test_videos = []
val_videos = []

for single_subdivision in subdivisions:
  train_indexes = single_subdivision[0]
  test_indexes = single_subdivision[1]
  val_videos.append(df_videos.iloc[train_indexes[:40]])
  train_videos.append(df_videos.iloc[train_indexes[40:]])
  test_videos.append(df_videos.iloc[test_indexes])

In [ ]:
# TRAIN
for i in range(0, 5):
  for index_i, row_i in train_videos[i].iterrows():
    dict_train = {'filename':[], 'Flames':[]}       # CREATE A NEW CSV FOR EACH VIDEO
    for index_j, row_j in starting_csv.iterrows():
      if '/' + row_i['Name video'] + '/' in row_j['filename']:
        dict_train['filename'].append('/user/dsilvitelli/kerasenv/bin' + row_j['filename'][23:])
        dict_train['Flames'].append(row_j['Flames'])
    df_train = pd.DataFrame.from_dict(dict_train)
    df_train.to_csv('csv/Train/wild/KFold_long-range/fold' + str(i+1) + '/' + row_i['Name video'] + '.csv')

# TEST
for i in range(0, 5):
  for index_i, row_i in test_videos[i].iterrows():
    dict_test = {'filename':[], 'Flames':[]}       # CREATE A NEW CSV FOR EACH VIDEO
    for index_j, row_j in starting_csv.iterrows():
      if '/' + row_i['Name video'] + '/' in row_j['filename']:
        dict_test['filename'].append('/user/dsilvitelli/kerasenv/bin' + row_j['filename'][23:])
        dict_test['Flames'].append(row_j['Flames'])
    df_test = pd.DataFrame.from_dict(dict_test)
    df_test.to_csv('csv/Test/wild/KFold_long-range/fold' + str(i+1) + '/' + row_i['Name video'] + '.csv')

# VAL
for i in range(0, 5):
  for index_i, row_i in val_videos[i].iterrows():
    dict_val = {'filename':[], 'Flames':[]}       # CREATE A NEW CSV FOR EACH VIDEO
    for index_j, row_j in starting_csv.iterrows():
      if '/' + row_i['Name video'] + '/' in row_j['filename']:
        dict_val['filename'].append('/user/dsilvitelli/kerasenv/bin' + row_j['filename'][23:])
        dict_val['Flames'].append(row_j['Flames'])
    df_val = pd.DataFrame.from_dict(dict_val)
    df_val.to_csv('csv/Validation/wild/KFold_long-range/fold' + str(i+1) + '/' + row_i['Name video'] + '.csv')

In [ ]:
# check subdivisions
to_check_test = []
to_check_train = []
for i in range(0, 5):
  data_path = os.path.join('csv', 'Test/wild/KFold_long-range/fold' + str(i+1))
  data_files = os.listdir(data_path)
  to_check_test.append(data_files)
for i in range(0, 5):
  data_path = os.path.join('csv', 'Train/wild/KFold_long-range/fold' + str(i+1))
  data_files = os.listdir(data_path)
  to_check_train.append(data_files)